## Computing Centrality Measures

In [1]:
import os
import pickle
from functools import partial
import pprint as pp

import pandas as pd

import pyrosm
import osmnx as ox

import networkx as nx

### Read in the osm.pbf file and create a NetworkX graph

In [5]:
# From https://pyrosm.readthedocs.io/en/latest/basics.html?highlight=osmnx#export-to-networkx-osmnx

# Initialize the reader
osm = pyrosm.OSM('./data/osm/albany-county_new-york-210101.osm.pbf')

# Get all walkable roads and the nodes 
nodes, edges = osm.get_network(nodes=True, network_type='driving')

# Export the nodes and edges to NetworkX graph
G = osm.to_graph(nodes, edges, graph_type="networkx", retain_all=True, osmnx_compatible=True)

In [3]:
def flatten(k, v):
    v = v[k] if isinstance(v[k], list) else [v[k]]
    v = [x for x in v if isinstance(x, str)]

    return v[0] if v else None

### Use OSMnx to simplify the network

In [4]:
# Our road network graph can be represented as two GeoDataFrames
ox_g = ox.simplify_graph(G)

# WARNING: Reindexing and exporting to GPKG throws the following error:
#            ValueError: cannot insert osmid, already exists
#
# The following workaround is suggested by the library's author:
#   https://github.com/gboeing/osmnx/issues/638#issuecomment-756948363
for node, data in ox_g.nodes(data=True):
    if 'osmid' in data:
        data['osmid_original'] = data.pop('osmid')

# NOTE: add_edge_speeds crashes without the following work around
for e in list(ox_g.edges(data=True)):
    if not (isinstance(e[2]['maxspeed'], str) or e[2]['maxspeed'] == None):
        e[2]['maxspeed'] = flatten('maxspeed', e[2])

In [5]:
# https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.routing.add_edge_speeds
ox.add_edge_speeds(ox_g)

# https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.routing.add_edge_travel_times
ox.add_edge_travel_times(ox_g)

In [ ]:
node_centrality_file = 'node_betweenness_centrality.albany_roadways_210101.pkl'

node_betweenness_centrality = None
try:
    with open(node_centrality_file, 'rb') as file:
        node_betweenness_centrality = pickle.load(file)
except FileNotFoundError:
    print('Must compute node betweenness centrality...')
    # Note: took 5h30m to run for Albany County
    node_betweenness_centrality = nx.betweenness_centrality(ox_g, weight='travel_time')
    with open(node_centrality_file, 'wb') as f:
        pickle.dump(node_betweenness_centrality, f)

In [7]:
edge_centrality_file = 'edge_betweenness_centrality.albany_roadways_210101.pkl'

try:
    with open(edge_centrality_file, 'rb') as file:
        edge_betweenness_centrality = pickle.load(file)
except FileNotFoundError:
    print('Must compute edge betweenness centrality...')
    # Note: took 5h30m to run for Albany County
    edge_betweenness_centrality = nx.edge_betweenness_centrality(ox_g, weight='travel_time')
    with open(edge_centrality_file, 'wb') as f:
        pickle.dump(edge_betweenness_centrality, f)

In [26]:

g = ox_g.copy()

for node, data in list(g.nodes(data=True)):
    data['centrality'] = node_betweenness_centrality[node]

for edge in list(g.edges(data=True)):
    edge[2]['centrality'] = edge_betweenness_centrality[(edge[0],edge[1], 0)]

In [30]:
output_file_name = 'albany_roadways_210101.with_betweenness_centrality.gpkg'

try:
    os.remove(output_file_name)
except OSError:
    pass

gdf_nodes, gdf_relationships = ox.graph_to_gdfs(g)

gdf_nodes.to_file(output_file_name, layer='intersections', driver='GPKG')
gdf_relationships.to_file(output_file_name, layer='roadways', driver='GPKG')